# Implicit Euler Integration Scheme

Credit to professor Bo Zhu, on whose COSC89 Course Notes the following math is based

Additional credit to @feisuzhu, on whose implicit_mass_spring.py code this is based

The code creates a "cloth", or a stiff ntwork of interconnected springs and nodes

To have the cloth interact with a bowl boundary, set line 6 to true: `bowl_boundary = True`\
To have the cloth interact with the screen edges themselves as the boundary (with the inclusion of a small, circular perturbation at the bottom, set line 6 to false

In [6]:
import argparse
import numpy as np
import taichi as ti

res = 700
bowl_boundary = False
bowl_r = 400
obstacle_r = 50.0
c = ti.Vector([0.5, 0.75]) # center of the bowl
r = 5 #particle radius


@ti.data_oriented
class Cloth:
    def __init__(self, N): 
        self.N = N
        self.NF = 2 * N**2  # number of faces
        self.NV = (N + 1)**2  # number of vertices
        self.NE = 2 * N * (N + 1) + 2 * N * N  # numbser of edges
        self.pos = ti.Vector.field(2, ti.f32, self.NV)
        self.initPos = ti.Vector.field(2, ti.f32, self.NV)
        self.vel = ti.Vector.field(2, ti.f32, self.NV)
        self.force = ti.Vector.field(2, ti.f32, self.NV)
        self.mass = ti.field(ti.f32, self.NV)

        self.spring = ti.Vector.field(2, ti.i32, self.NE)
        self.indices = ti.field(ti.i32, 2 * self.NE)
        self.Jx = ti.Matrix.field(2, 2, ti.f32,
                                  self.NE)  # Jacobian with respect to position
        self.Jv = ti.Matrix.field(2, 2, ti.f32,
                                  self.NE)  # Jacobian with respect to velocity
        self.rest_len = ti.field(ti.f32, self.NE)
        self.ks = 2000.0  # spring stiffness
        self.kd = 0.5  # damping constant
        self.ks_p = 1e2   # spring stiffness between particles and boundary
        self.kd_p = 1e1 # damping constant between particles and boundary
        self.ks_boundary = 2000.0
        self.kd_boundary = 1

        self.gravity = ti.Vector([0.0, -2.0])
        self.init_pos()
        self.init_edges()
        self.MassBuilder = ti.linalg.SparseMatrixBuilder(
            2 * self.NV, 2 * self.NV, max_num_triplets=10000)
        self.DBuilder = ti.linalg.SparseMatrixBuilder(2 * self.NV,
                                                      2 * self.NV,
                                                      max_num_triplets=10000)
        self.KBuilder = ti.linalg.SparseMatrixBuilder(2 * self.NV,
                                                      2 * self.NV,
                                                      max_num_triplets=10000)
        self.init_mass_sp(self.MassBuilder)
        self.M = self.MassBuilder.build()
        self.fix_vertex = [self.N, self.NV - 1]

    @ti.kernel
    def init_pos(self):
        # add initial velocity
        vx_0 = -0.5
        for i, j in ti.ndrange(self.N + 1, self.N + 1):
            k = i * (self.N + 1) + j
            self.pos[k] = ti.Vector([i, j]) / self.N * 0.4 + ti.Vector(
                [0.25, 0.35])
            self.initPos[k] = self.pos[k]
            self.vel[k] = ti.Vector([vx_0 + (ti.random()-0.5)*0.8, 0])
            # self.vel[k] = ti.Vector([0, 0])
            self.mass[k] = 1.0

    @ti.kernel
    def init_edges(self):
        pos, spring, N, rest_len = ti.static(self.pos, self.spring, self.N,
                                             self.rest_len)
        for i, j in ti.ndrange(N + 1, N):
            idx, idx1 = i * N + j, i * (N + 1) + j
            spring[idx] = ti.Vector([idx1, idx1 + 1])
            rest_len[idx] = (pos[idx1] - pos[idx1 + 1]).norm()
        start = N * (N + 1)
        for i, j in ti.ndrange(N, N + 1):
            idx, idx1, idx2 = start + i + j * N, i * (N + 1) + j, i * (
                N + 1) + j + N + 1
            spring[idx] = ti.Vector([idx1, idx2])
            rest_len[idx] = (pos[idx1] - pos[idx2]).norm()
        start = 2 * N * (N + 1)
        for i, j in ti.ndrange(N, N):
            idx, idx1, idx2 = start + i * N + j, i * (N + 1) + j, (i + 1) * (
                N + 1) + j + 1
            spring[idx] = ti.Vector([idx1, idx2])
            rest_len[idx] = (pos[idx1] - pos[idx2]).norm()
        start = 2 * N * (N + 1) + N * N
        for i, j in ti.ndrange(N, N):
            idx, idx1, idx2 = start + i * N + j, i * (N + 1) + j + 1, (
                i + 1) * (N + 1) + j
            spring[idx] = ti.Vector([idx1, idx2])
            rest_len[idx] = (pos[idx1] - pos[idx2]).norm()

    @ti.kernel
    def init_mass_sp(self, M: ti.types.sparse_matrix_builder()):
        for i in range(self.NV):
            mass = self.mass[i]
            M[2 * i + 0, 2 * i + 0] += mass
            M[2 * i + 1, 2 * i + 1] += mass

    @ti.func
    def clear_force(self):
        for i in self.force:
            self.force[i] = ti.Vector([0.0, 0.0])

    @ti.func
    def check_boundaries_bowl(self):
        ### treating circles as point masses ###
        z = ti.Vector([0.0, 0.0])
            
        for idx in ti.ndrange(self.NV):
            # boundary check with the bowl
            phi = bowl_r - ((self.pos[idx] - c).norm() * res) - r
            neg_grad = (self.pos[idx] - c).normalized()
            if phi < 0: # outside of the bowl
                self.force[idx] += self.ks_p * (phi) * neg_grad
                self.force[idx] += self.kd_p * (ti.Vector([0, 0]) - self.vel[idx]).dot(neg_grad) * neg_grad

            for idx2 in range(idx+1, self.NV):
                # check for collision
                phi = (self.pos[idx] - self.pos[idx2]).norm() - r/res - r/res
                normal = (self.pos[idx2] - self.pos[idx]).normalized()
                if phi < 0:
                    force_spring = self.ks_p * (phi) * normal
                    force_damp = self.kd_p * (self.vel[idx] - self.vel[idx]).dot(normal) * normal
                    self.force[idx] += force_spring
                    self.force[idx2] -= force_spring
                    self.force[idx] += force_damp
                    self.force[idx2] -= force_damp
         
    
    @ti.func
    def check_boundaries_square(self):
        ### treating circles as point masses ###
        z = ti.Vector([0.0, 0.0])
            
        for idx in ti.ndrange(self.NV):
            # if the particle collides with lower boundary
            y = self.pos[idx][1]
            x = self.pos[idx][0]
            if y < 0: 
                phi = ti.Vector([0, 1.0])
                dis_y = 0 - self.pos[idx][1]
                dis = ti.Vector([0, dis_y])
                force_spring = self.ks_boundary * (dis.norm()) * phi
                force_damp = self.kd_boundary * ((z - self.vel[idx]).dot(phi)) * phi
                
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
            # if it collides with upper boundary
            elif y > 1:
                phi = ti.Vector([0, -1.0])
                dis_y = 1 - self.pos[idx][1]
                dis = ti.Vector([0, dis_y])
                force_spring = self.ks_boundary * (dis.norm()) * phi
                force_damp = self.kd_boundary * ((z - self.vel[idx]).dot(phi)) * phi
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
            # left boundary
            if x < 0:
                phi = ti.Vector([1.0, 0])
                dis_x = 0 - self.pos[idx][0]
                dis = ti.Vector([dis_x, 0])
                force_spring = self.ks_boundary * (dis.norm()) * phi
                force_damp = self.kd_boundary * ((z - self.vel[idx]).dot(phi)) * phi
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
            # right boundary
            elif x > 1:
                phi = ti.Vector([-1.0, 0])
                dis_x = 1 - self.pos[idx][0]
                dis = ti.Vector([dis_x, 0])
                force_spring = self.ks_boundary * (dis.norm()) * phi
                force_damp = self.kd_boundary * ((z - self.vel[idx]).dot(phi)) * phi
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
            # collide with circle
            dispFromCenter = self.pos[idx] - ti.Vector([0.5, 0]) # displacement from center of circle
            if dispFromCenter.norm() < (obstacle_r/res):
                r = dispFromCenter / dispFromCenter.norm() * (1.0/5.0)
                disp = -(r - dispFromCenter)
                phi = disp.norm()
                force_spring = self.ks_boundary * phi * (-disp.normalized())
                force_damp = self.kd_boundary * ((z - self.vel[idx]).dot(-disp.normalized())) * (-disp.normalized())
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
                
    
    @ti.kernel
    def compute_force(self):
        self.clear_force()
        for i in self.force:
            self.force[i] += self.gravity * self.mass[i]

        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            pos1, pos2 = self.pos[idx1], self.pos[idx2]
            dis = pos2 - pos1
            force = self.ks * (dis.norm() -
                               self.rest_len[i]) * dis.normalized()
            self.force[idx1] += force
            self.force[idx2] -= force
        if (bowl_boundary):
            self.check_boundaries_bowl()
        else:
            self.check_boundaries_square()
    

    @ti.kernel
    def compute_Jacobians(self):
        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            pos1, pos2 = self.pos[idx1], self.pos[idx2]
            dx = pos1 - pos2
            I = ti.Matrix([[1.0, 0.0], [0.0, 1.0]])
            dxtdx = ti.Matrix([[dx[0] * dx[0], dx[0] * dx[1]],
                               [dx[1] * dx[0], dx[1] * dx[1]]])
            l = dx.norm()
            if l != 0.0:
                l = 1.0 / l
            self.Jx[i] = (I - self.rest_len[i] * l *
                          (I - dxtdx * l**2)) * self.ks
            self.Jv[i] = self.kd * I
            

    @ti.kernel
    def assemble_K(self, K: ti.types.sparse_matrix_builder()):
        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            for m, n in ti.static(ti.ndrange(2, 2)):
                K[2 * idx1 + m, 2 * idx1 + n] -= self.Jx[i][m, n]
                K[2 * idx1 + m, 2 * idx2 + n] += self.Jx[i][m, n]
                K[2 * idx2 + m, 2 * idx1 + n] += self.Jx[i][m, n]
                K[2 * idx2 + m, 2 * idx2 + n] -= self.Jx[i][m, n]


    @ti.kernel
    def assemble_D(self, D: ti.types.sparse_matrix_builder()):
        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            for m, n in ti.static(ti.ndrange(2, 2)):
                D[2 * idx1 + m, 2 * idx1 + n] -= self.Jv[i][m, n]
                D[2 * idx1 + m, 2 * idx2 + n] += self.Jv[i][m, n]
                D[2 * idx2 + m, 2 * idx1 + n] += self.Jv[i][m, n]
                D[2 * idx2 + m, 2 * idx2 + n] -= self.Jv[i][m, n]


    @ti.kernel
    def updatePosVel(self, h: ti.f32, dv: ti.types.ndarray()):
        for i in self.pos:
            self.vel[i] += ti.Vector([dv[2 * i], dv[2 * i + 1]])
            self.pos[i] += h * self.vel[i]

    def update(self, h):
        self.compute_force()

        self.compute_Jacobians()
        # Assemble global system

        self.assemble_D(self.DBuilder)
        D = self.DBuilder.build()

        self.assemble_K(self.KBuilder)
        K = self.KBuilder.build()

        A = self.M - h * D - h**2 * K

        vel = self.vel.to_numpy().reshape(2 * self.NV)
        force = self.force.to_numpy().reshape(2 * self.NV)
        b = (force + h * K @ vel) * h
        # Sparse solver
        solver = ti.linalg.SparseSolver(solver_type="LDLT")
        solver.analyze_pattern(A)
        solver.factorize(A)
        # Solve the linear system
        dv = solver.solve(b)
        self.updatePosVel(h, dv)

    def display(self, gui, radius=5, color=0xffffff):
        lines = self.spring.to_numpy()
        pos = self.pos.to_numpy()
        edgeBegin = np.zeros(shape=(lines.shape[0], 2))
        edgeEnd = np.zeros(shape=(lines.shape[0], 2))
        for i in range(lines.shape[0]):
            idx1, idx2 = lines[i][0], lines[i][1]
            edgeBegin[i] = pos[idx1]
            edgeEnd[i] = pos[idx2]
        gui.lines(edgeBegin, edgeEnd, radius=2, color=0x0000ff)
        gui.circles(self.pos.to_numpy(), radius, color)
        # draw boundary circle at bottom
        if not bowl_boundary:
            gui.circle((0.5, 0), radius=obstacle_r)


def main():
    ti.init(arch=ti.cpu)
    h = 0.01
    cloth = Cloth(N=5)
    ctr = 0
    save_pic = False


    gui = ti.GUI('Implicit Mass Spring System', res=(res, res))
    while gui.running:
        for e in gui.get_events():
            if e.key == gui.ESCAPE:
                gui.running = False
            elif e.key == 'c':
                save_pic = True

        cloth.update(h)

        cloth.display(gui, radius = r)
        
        if save_pic:
            filename = f'implicit_spring_vis_{ctr:02d}.png'
            gui.show(filename)
            ctr+=1
            save_pic = False
        else:
            gui.show()

if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64
